# TraceNet - Custom Facial Recognition Training

**Final Year Project: Advanced Face Recognition System**

This notebook trains a custom facial recognition model from scratch:
- **Architecture:** ResNet-50 inspired (TraceNet)
- **Loss:** ArcFace (Angular Margin Loss)
- **Dataset:** LFW (Labeled Faces in the Wild)
- **Target Accuracy:** 95%+ on LFW benchmark

---

## Setup Instructions:
1. **Runtime → Change runtime type → GPU (T4 or better)**
2. Run all cells in order
3. Training will take ~6-12 hours
4. Models are saved to Google Drive automatically

## 1. Environment Setup

In [39]:
# Mount Google Drive (to save checkpoints)
from google.colab import drive
drive.mount('/content/drive')

# Create project directory
!mkdir -p /content/drive/MyDrive/TraceNet
!mkdir -p /content/drive/MyDrive/TraceNet/checkpoints
!mkdir -p /content/drive/MyDrive/TraceNet/logs

print("✅ Google Drive mounted successfully!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive mounted successfully!


In [40]:
# Install dependencies
!pip install -q torch torchvision torchaudio
!pip install -q opencv-python-headless
!pip install -q pillow
!pip install -q albumentations
!pip install -q scikit-learn
!pip install -q matplotlib
!pip install -q tqdm
!pip install -q wandb  # Optional: for experiment tracking

print("✅ All dependencies installed!")

✅ All dependencies installed!


In [41]:
# Check GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
else:
    print("⚠️ WARNING: No GPU detected! Go to Runtime → Change runtime type → GPU")

PyTorch version: 2.9.0+cu128
CUDA available: True
GPU: Tesla T4
CUDA version: 12.8


In [42]:
import shutil
import os
if os.path.exists('/content/lfw'):
    print("🗑️ Removing previous incomplete download...")
    shutil.rmtree('/content/lfw')
    print("✅ Cleanup complete!")
if os.path.exists('/content/data'):
    print("🗑️ Removing previous data directory...")
    shutil.rmtree('/content/data')
    print("✅ Cleanup complete!")
# ====================
# STEP 2: Install kagglehub
# ====================
!pip install -q kagglehub

🗑️ Removing previous incomplete download...
✅ Cleanup complete!
🗑️ Removing previous data directory...
✅ Cleanup complete!


## 2. Download & Prepare Dataset

**LFW Dataset:**
- 13,233 images
- 5,749 identities
- Perfect for training and benchmarking

In [43]:
# Install kagglehub
!pip install -q kagglehub

import kagglehub
import os
import shutil

# Download LFW dataset from Kaggle
if not os.path.exists('/content/lfw'):
    print("Downloading LFW dataset from Kaggle...")
    path = kagglehub.dataset_download("jessicali9530/lfw-dataset")

    print(f"✅ Dataset downloaded to: {path}")

    # The dataset has a subfolder 'lfw-deepfunneled' with the actual images
    source = os.path.join(path, 'lfw-deepfunneled')

    # Copy (not move) because Kaggle cache is read-only
    print(f"Copying dataset from {source} to /content/lfw...")
    shutil.copytree(source, '/content/lfw')

    print("✅ LFW dataset ready at /content/lfw")
else:
    print("✅ LFW dataset already available!")

# Verify
print(f"\nTotal images: {sum([len(files) for r, d, files in os.walk('/content/lfw')])}")

Using Colab cache for faster access to the 'lfw-dataset' dataset.
✅ Dataset downloaded to: /kaggle/input/lfw-dataset
Copying dataset from /kaggle/input/lfw-dataset/lfw-deepfunneled to /content/lfw...
✅ LFW dataset ready at /content/lfw

Total images: 13233


## 3. Model Architecture - TraceNet

In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResidualBlock(nn.Module):
    """Residual block with skip connection"""
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, stride, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 1, stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        return F.relu(out)


class TraceNet(nn.Module):
    """Custom Face Recognition Network (ResNet-50 inspired)"""
    def __init__(self, embedding_size=512):
        super().__init__()

        # Initial convolution
        self.conv1 = nn.Conv2d(3, 64, 7, 2, 3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.maxpool = nn.MaxPool2d(3, 2, 1)

        # Residual layers
        self.layer1 = self._make_layer(64, 64, 3, 1)
        self.layer2 = self._make_layer(64, 128, 4, 2)
        self.layer3 = self._make_layer(128, 256, 6, 2)
        self.layer4 = self._make_layer(256, 512, 3, 2)

        # Global average pooling
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        # Embedding layer
        self.fc = nn.Linear(512, embedding_size)
        self.bn_fc = nn.BatchNorm1d(embedding_size)

        # Initialize weights
        self._initialize_weights()

    def _make_layer(self, in_channels, out_channels, blocks, stride):
        layers = []
        layers.append(ResidualBlock(in_channels, out_channels, stride))
        for _ in range(1, blocks):
            layers.append(ResidualBlock(out_channels, out_channels, 1))
        return nn.Sequential(*layers)

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.maxpool(F.relu(self.bn1(self.conv1(x))))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.bn_fc(self.fc(x))
        return x


class ArcFaceLoss(nn.Module):
    """ArcFace: Additive Angular Margin Loss"""
    def __init__(self, embedding_size, num_classes, s=64.0, m=0.50):
        super().__init__()
        self.s = s  # Scale
        self.m = m  # Margin
        self.weight = nn.Parameter(torch.FloatTensor(num_classes, embedding_size))
        nn.init.xavier_uniform_(self.weight)

    def forward(self, embeddings, labels):
        # Normalize
        embeddings = F.normalize(embeddings, p=2, dim=1)
        weight_norm = F.normalize(self.weight, p=2, dim=1)

        # Cosine similarity
        cos_theta = F.linear(embeddings, weight_norm).clamp(-1, 1)

        # Add margin
        theta = torch.acos(cos_theta)
        one_hot = torch.zeros_like(cos_theta)
        one_hot.scatter_(1, labels.view(-1, 1), 1)

        theta_m = theta + self.m * one_hot
        output = torch.cos(theta_m) * self.s

        return output

print("✅ Model architecture defined!")

✅ Model architecture defined!


In [45]:
import shutil
from pathlib import Path
import random
from tqdm import tqdm
import os

def prepare_dataset():
    lfw_dir = Path("/content/lfw")
    data_dir = Path("/content/data")

    # Ensure previous (potentially empty) data directory is removed
    if data_dir.exists():
        print("🗑️ Removing previous data directory to re-prepare...")
        shutil.rmtree(data_dir)
        print("✅ Cleanup complete!")

    # Create directories
    (data_dir / "train").mkdir(parents=True, exist_ok=True)
    (data_dir / "val").mkdir(parents=True, exist_ok=True)
    (data_dir / "test").mkdir(parents=True, exist_ok=True)

    # Get all identities with at least 5 images
    identities = []
    for person_dir in lfw_dir.iterdir():
        if person_dir.is_dir():
            images = list(person_dir.glob("*.jpg"))
            if len(images) >= 5: # Changed from 10 to 5
                identities.append((person_dir.name, images))

    print(f"Found {len(identities)} identities with 5+ images")

    # Split: 70% train, 15% val, 15% test
    random.shuffle(identities)
    train_split = int(0.7 * len(identities))
    val_split = int(0.85 * len(identities))

    splits = {
        'train': identities[:train_split],
        'val': identities[train_split:val_split],
        'test': identities[val_split:]
    }

    for split_name, split_data in splits.items():
        for person_name, images in tqdm(split_data, desc=f"Preparing {split_name}"):
            person_dir = data_dir / split_name / person_name
            person_dir.mkdir(exist_ok=True)

            for img in images:
                shutil.copy(img, person_dir / img.name)

    print("\n✅ Dataset prepared!")
    print(f"   Train: {len(splits['train'])} identities")
    print(f"   Val: {len(splits['val'])} identities")
    print(f"   Test: {len(splits['test'])} identities")

prepare_dataset()

Found 0 identities with 5+ images


Preparing train: 0it [00:00, ?it/s]
Preparing val: 0it [00:00, ?it/s]
Preparing test: 0it [00:00, ?it/s]


✅ Dataset prepared!
   Train: 0 identities
   Val: 0 identities
   Test: 0 identities


## 4. Dataset & DataLoader

In [46]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os

class FaceDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []

        # Load all images
        for idx, person in enumerate(sorted(os.listdir(root_dir))):
            person_dir = os.path.join(root_dir, person)
            if not os.path.isdir(person_dir):
                continue

            for img_name in os.listdir(person_dir):
                if img_name.endswith(('.jpg', '.jpeg', '.png')):
                    img_path = os.path.join(person_dir, img_name)
                    self.samples.append((img_path, idx))

        self.num_classes = len(set([s[1] for s in self.samples]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]

        try:
            image = Image.open(img_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
            return image, label
        except Exception as e:
            # Return next sample if error
            return self.__getitem__((idx + 1) % len(self))

# Data transforms
train_transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

val_transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Create datasets
train_dataset = FaceDataset('/content/data/train', transform=train_transform)
val_dataset = FaceDataset('/content/data/val', transform=val_transform)

print(f"✅ Datasets created!")
print(f"   Train: {len(train_dataset)} images, {train_dataset.num_classes} identities")
print(f"   Val: {len(val_dataset)} images, {val_dataset.num_classes} identities")

✅ Datasets created!
   Train: 0 images, 0 identities
   Val: 0 images, 0 identities


In [47]:
# Create data loaders
BATCH_SIZE = 64  # Adjust based on GPU memory

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

print(f"✅ DataLoaders created!")
print(f"   Batches per epoch: {len(train_loader)}")

ValueError: num_samples should be a positive integer value, but got num_samples=0

## 5. Training Setup

In [ ]:
# Training configuration
EMBEDDING_SIZE = 512
LEARNING_RATE = 0.001
NUM_EPOCHS = 100
CHECKPOINT_DIR = '/content/drive/MyDrive/TraceNet/checkpoints'

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Model
model = TraceNet(embedding_size=EMBEDDING_SIZE).to(device)
criterion = ArcFaceLoss(
    embedding_size=EMBEDDING_SIZE,
    num_classes=train_dataset.num_classes,
    s=64.0,
    m=0.50
).to(device)

# Optimizer & Scheduler
optimizer = torch.optim.Adam(
    list(model.parameters()) + list(criterion.parameters()),
    lr=LEARNING_RATE
)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=NUM_EPOCHS
)

print(f"\n✅ Training setup complete!")
print(f"   Model parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"   Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

## 6. Training Loop

In [ ]:
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

# Training history
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': []
}

best_val_acc = 0.0

def train_epoch(epoch):
    model.train()
    criterion.train()

    total_loss = 0
    correct = 0
    total = 0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}")

    for images, labels in pbar:
        images = images.to(device)
        labels = labels.to(device)

        # Forward
        embeddings = model(images)
        outputs = criterion(embeddings, labels)
        loss = F.cross_entropy(outputs, labels)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Statistics
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        # Update progress
        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{100.*correct/total:.2f}%'
        })

    avg_loss = total_loss / len(train_loader)
    accuracy = 100. * correct / total

    return avg_loss, accuracy


def validate():
    model.eval()
    criterion.eval()

    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validating"):
            images = images.to(device)
            labels = labels.to(device)

            embeddings = model(images)
            outputs = criterion(embeddings, labels)
            loss = F.cross_entropy(outputs, labels)

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    avg_loss = total_loss / len(val_loader)
    accuracy = 100. * correct / total

    return avg_loss, accuracy


def save_checkpoint(epoch, val_acc, is_best=False):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'criterion_state_dict': criterion.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'val_accuracy': val_acc,
        'history': history
    }

    # Save latest
    torch.save(checkpoint, f'{CHECKPOINT_DIR}/latest.pth')

    # Save best
    if is_best:
        torch.save(checkpoint, f'{CHECKPOINT_DIR}/best.pth')
        print(f"✅ New best model saved! Accuracy: {val_acc:.2f}%")


def plot_history():
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Loss
    ax1.plot(history['train_loss'], label='Train')
    ax1.plot(history['val_loss'], label='Validation')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.set_title('Training & Validation Loss')
    ax1.legend()
    ax1.grid(True)

    # Accuracy
    ax2.plot(history['train_acc'], label='Train')
    ax2.plot(history['val_acc'], label='Validation')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy (%)')
    ax2.set_title('Training & Validation Accuracy')
    ax2.legend()
    ax2.grid(True)

    plt.tight_layout()
    plt.savefig(f'{CHECKPOINT_DIR}/../logs/training_history.png')
    plt.show()

print("✅ Training functions defined!")

In [ ]:
# Main training loop
print("🚀 Starting training...\n")

for epoch in range(NUM_EPOCHS):
    # Train
    train_loss, train_acc = train_epoch(epoch)

    # Validate
    val_loss, val_acc = validate()

    # Update scheduler
    scheduler.step()

    # Save history
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)

    # Print summary
    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}:")
    print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
    print(f"  LR: {optimizer.param_groups[0]['lr']:.6f}")

    # Save checkpoint
    is_best = val_acc > best_val_acc
    if is_best:
        best_val_acc = val_acc
    save_checkpoint(epoch, val_acc, is_best)

    # Plot progress every 10 epochs
    if (epoch + 1) % 10 == 0:
        plot_history()

print(f"\n🎉 Training completed!")
print(f"Best validation accuracy: {best_val_acc:.2f}%")

## 7. Final Evaluation & Export

In [ ]:
# Load best model
checkpoint = torch.load(f'{CHECKPOINT_DIR}/best.pth')
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"✅ Best model loaded (Epoch {checkpoint['epoch']+1})")
print(f"   Validation Accuracy: {checkpoint['val_accuracy']:.2f}%")

In [ ]:
# Export for deployment (model only, no loss)
deployment_model = {
    'model_state_dict': model.state_dict(),
    'embedding_size': EMBEDDING_SIZE,
    'accuracy': checkpoint['val_accuracy'],
    'epoch': checkpoint['epoch']
}

torch.save(deployment_model, f'{CHECKPOINT_DIR}/tracenet_deployment.pth')
print("✅ Deployment model exported to Google Drive!")
print(f"   Path: {CHECKPOINT_DIR}/tracenet_deployment.pth")

In [ ]:
# Test inference
import numpy as np

def extract_embedding(image):
    """Extract embedding from image"""
    model.eval()
    with torch.no_grad():
        if isinstance(image, np.ndarray):
            image = Image.fromarray(image)
        image_tensor = val_transform(image).unsqueeze(0).to(device)
        embedding = model(image_tensor)
        embedding = F.normalize(embedding, p=2, dim=1)
    return embedding.cpu().numpy()[0]

# Test with sample image
test_img_path = train_dataset.samples[0][0]
test_img = Image.open(test_img_path).convert('RGB')
embedding = extract_embedding(test_img)

print(f"✅ Embedding extracted successfully!")
print(f"   Shape: {embedding.shape}")
print(f"   L2 Norm: {np.linalg.norm(embedding):.4f} (should be ~1.0)")

## 8. Download Model to Local System

The trained model is saved in your Google Drive at:
- **Best model:** `/content/drive/MyDrive/TraceNet/checkpoints/best.pth`
- **Deployment model:** `/content/drive/MyDrive/TraceNet/checkpoints/tracenet_deployment.pth`

You can download it directly from Google Drive or use the code below:

In [ ]:
from google.colab import files

# Download deployment model
files.download(f'{CHECKPOINT_DIR}/tracenet_deployment.pth')
print("✅ Model download started!")

## 9. Integration Instructions

To use this model in your TraceIQ backend:

1. Download `tracenet_deployment.pth` to your project:
   ```bash
   cp tracenet_deployment.pth backend/models/
   ```

2. Load the model in your inference service:
   ```python
   checkpoint = torch.load('models/tracenet_deployment.pth')
   model = TraceNet(embedding_size=512)
   model.load_state_dict(checkpoint['model_state_dict'])
   model.eval()
   ```

3. Extract embeddings:
   ```python
   embedding = model(preprocessed_image)
   embedding = F.normalize(embedding, p=2, dim=1)
   ```

**Expected Performance:**
- Validation Accuracy: `{best_val_acc:.2f}%`
- Inference Time: ~5-10ms per face (GPU)
- Embedding Size: 512 dimensions

---

**🎓 Congratulations! You've successfully trained your custom facial recognition model!**